In [5]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np
from network_construction.network import source_treatment, get_Utown

import sys
sys.path.append("C:\\Users\mbans\Desktop\CMOR492-DWS\DWS")

# sys.path.append('/Users/danielsuarez/Documents/Academic/Spring2025/SeniorDesign/CMOR492-DWS/')

In [6]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G)

In [1]:
# USER SET PARAMETERS
SOURCE_PROD = 0.17
TREATMENT_CAP = 2 * SOURCE_PROD * len(source_nodes)

NameError: name 'source_nodes' is not defined

In [8]:
### MODEL PARAMETERS

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')

LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v

D = [0.2, 0.3, 0.4, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe

SR = {}  # Production at Source node i
for node in source_nodes:
    G.nodes[node]['production'] = SOURCE_PROD # Per Minute
    SR[node] = SOURCE_PROD

CAP = {}  # Capacity at treatment node j
for node in treatment_nodes:
    G.nodes[node]['capacity'] = TREATMENT_CAP
    CAP[node] = TREATMENT_CAP

Vmin = 0.6 * 60
Vmax = 3 * 60 # (m/s) * (60s/min.)

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment

CT = 1000000  # Cost of trucking (recourse)
M = 1e7  # Arbitrarily Large Number

Smin = 0.01  # Min slope grade
Smax = 0.1  # Max slope grade
W = 0.5  # (m) Buffer Width


In [9]:
m = gp.Model()

### DECISION VARIABLES

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name='x')  # Path ij used
y = m.addVars(treatment_nodes, vtype=GRB.BINARY, name='y')  # treatment at node j
z = m.addVars(G.edges, vtype=GRB.BINARY, name='z')  # edge e used

d = m.addVars(G.edges, D, vtype=GRB.BINARY, name='d')  # Pipe size s at edge e


# Recourse Amount
r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # flow handled at trucking at edge e

# Flow in Edge e
Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e

# Node Elevation
el = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='el')  # Elevation at node el_v

# Path Flow
p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name='p')

c = m.addVars(G.nodes, vtype=GRB.BINARY, name='c')

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-17


In [10]:
# UNCOMMENT IF YOU WANT TO MODEL A CENTRALIZED SYSTEM
# centralized = m.addConstr(gp.quicksum(y[j] for j in treatment_nodes) == 1, name='centralized system')
# m.update()

In [11]:
### CONSTRAINTS

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs((p[i, j] >= (SR[i] * x[i, j]) - r[i] for i, j in Path.keys()), name='node_prod_rec')

# TREATMENT CAPACITY
treat_cap = m.addConstrs((gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j] for j in treatment_nodes), name='treat_cap')

#  NODE ASSIGNMENT
node_assign = m.addConstrs((gp.quicksum(x[i, j] for j in treatment_nodes) == 1 for i in source_nodes), name='node_assign')

# PIPE SIZING
pipe_sizing = m.addConstrs((gp.quicksum(d[*e, s] for s in D) == z[e] for e in G.edges), name='pipe_sizing')  # ALWAYS BE SURE TO EXPAND e

# TODO: Go through this with John
# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs((Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges), name='flow_def')


# MIN/MAX SLOPE
min_slope = m.addConstrs((el[e[0]] - el[e[1]] >= (LE[e] * Smin) - (M * (1 - z[e])) for e in G.edges), name='min_slope')
max_slope = m.addConstrs((el[e[0]] - el[e[1]] <= (LE[e] * Smax) + (M * (1 - z[e])) for e in G.edges), name='max_slope')

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs((Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (d[*e, s]) for s in D) for e in G.edges), name='flow_vel')

# PIPES UNDERGROUND
underground = m.addConstrs((el[u] <= EL[u] for u in G.nodes), name='underground')
m.update()
# EDGE ACTIVATION

# TODO: Go through this with John 2
# EDGE ACTIVATION
ePath = {}  # Use this for Edge Activiation Constraint
for e, p in Path.items():
    ePath[e] = [(p[l - 1], p[l]) for l in range(1, len(p))]

edge_activate = m.addConstrs((gp.quicksum(z[e] for e in ePath[i, j]) >= NLinks[i, j] * x[i, j] for i, j in Path), name='edge_activate')


# ENVELOPES FOR MANNING

T = 11.9879
P = lambda LE, s: LE / (T * (s**(16/3)))
Qmax = lambda s: Vmax * ((np.pi / 8) * (s**2))


alpha = m.addVars(G.edges, D, lb=0, name='alpha')
beta = m.addVars(G.edges, D, lb=0, name='beta')


alpha_2 = m.addConstrs((alpha[*e, s] >= Q[e] + d[*e, s] * Qmax(s) - ( Qmax(s)) for e in G.edges for s in D), name='alpha_2')
alpha_3 = m.addConstrs((alpha[*e, s] <= Qmax(s) * d[*e, s] for e in G.edges for s in D), name='alpha_3')
alpha_4 = m.addConstrs((alpha[*e, s] <= Q[e] for e in G.edges for s in D), name='alpha_4')
alpha_5 = m.addConstrs((alpha[*e, s] <= Qmax(s) for e in G.edges for s in D), name='alpha_5')

beta_2 = m.addConstrs((beta[*e, s] >= (Qmax(s) * Q[e]) + (Qmax(s) * alpha[*e, s]) - (Qmax(s)**2) for e in G.edges for s in D), name='beta_2')
beta_3 = m.addConstrs((beta[*e, s] <= Qmax(s) * alpha[*e, s] for e in G.edges for s in D), name='beta_3')
beta_4 = m.addConstrs((beta[*e, s] <= Qmax(s) * Q[e] for e in G.edges for s in D), name='beta_4')

# manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= 0 for e in G.edges), name='manning_2')

# ADDED THE BIG M THING HERE BUT IDK IF IT COULD BE IMPROVED
manning_2 = m.addConstrs((el[e[1]] - el[e[0]] + gp.quicksum(P(LE[e], s) * beta[*e, s] for s in D) <= (1-z[e])*M for e in G.edges), name='manning_2')
m.update()

In [12]:
# OBJECTIVE EPXR 1: TREATMENT COSTS

treat_cost = gp.LinExpr()
for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j])

# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u]) + (EL[v] - el[v])) / 2) * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))

# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost_f = lambda u, v: gp.LinExpr(CB * LE[u, v] * gp.quicksum(s + ((2*W) * d[u, v, s]) for s in D))
# OBJECTIVE EXPR 4: PIPE COSTS
pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v] * gp.quicksum(CP[s] * d[u, v, s] for s in D))

excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v in G.edges)

# OBJECTIVE EXPR 5: RECOURSE TRUCKING

rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])

m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

# m.setObjective(0, GRB.MINIMIZE)

m.update()
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")

Model has 21955 variables and 31856 constraints.


In [ ]:
# m.write("singleperiod_nocontext2.lp")
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 31856 rows, 21955 columns and 268136 nonzeros
Model fingerprint: 0x150ac239
Model has 4036 quadratic objective terms
Variable types: 12040 continuous, 9915 integer (9915 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+07]
  Objective range  [2e+01, 5e+06]
  QObjective range [3e+02, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve removed 12039 rows and 3598 columns
Presolve time: 2.41s
Presolved: 23637 rows, 22177 columns, 248162 nonzeros
Variable types: 12623 continuous, 9554 integer (9554 binary)
Found heuristic solution: objective 1.804095e+07

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing primal log only...

Root relaxati

In [8]:
for v in r:
    if r[v].X > 0:
        print(r[v].VarName, r[v].X)

In [ ]:
m.computeIIS()

for c in m.getConstrs():
    if c.IISConstr:
        print(f"Constraint {c.ConstrName} is in the IIS")

for v in m.getVars():
    if v.IISLB > 0:
        print(f"Lower bound of {v.VarName} is in the IIS")
    elif v.IISUB > 0:
        print(f"Upper bound of {v.VarName} is in the IIS")


In [ ]:
for v in m.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

In [44]:
x_0 = {str(xv) : x[xv].X for xv in x}
y_0 = {yv : y[yv].X for yv in y}
z_0 = {str(zv) : z[zv].X for zv in z}

d_0 = {str(dv) : d[dv].X for dv in d}
el_0 = {elv: el[elv].X for elv in el}

In [45]:
import json

In [46]:
with open("context/x_sol.json", "w") as f:
    json.dump(x_0, f)

In [47]:
with open("context/z_sol.json", "w") as f:
    json.dump(z_0, f)

with open("context/d_sol.json", "w") as f:
    json.dump(d_0, f)

with open("context/el_sol.json", "w") as f:
    json.dump(el_0, f)

In [ ]:
# we should have a class for model output--that'll make both the multi-period recursion thing quicker and interpretation / visualization easier